In [12]:
import os
import os.path
import shutil
import csv
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

folder_path = '../data/classif_style2'
old_folder = '../data/train_2'

images = [f for f in os.listdir(old_folder) if os.path.isfile(os.path.join(old_folder, f)) and f.endswith('.jpg')]
df = pd.read_csv('../data/train_info.csv')

In [13]:
def train_test(df, images, folder_path, train_ratio):
    # only keep the images
    df = df[df['filename'].isin(images)]
    
    # simplify style name
    df['style'] = [str(styl).strip().replace(' ', '_').lower() for styl in df['style']]
    
    # filter out with not enough paintings
    df['nb_tot'] = df.groupby(['style'])['filename'].transform('count')
    # pour train_1: to_keep = list(df.sort_values(by = 'nb_tot', ascending = False)['style'].unique()[:15])
    # pour train_2: 
    to_keep = ['impressionism', 'cubism']
    df = df[df['style'].isin(to_keep)].reset_index(drop = True)
    
    # create folder name in function of train/test split
    df['ind'] = df.groupby(['style']).cumcount()
    df['train_test'] = [folder_path + '/train/' if ind/tot < train_ratio else folder_path +'/test/' \
                            for (ind,tot) in zip(df['ind'],df['nb_tot']) ]
    df['folder_name'] = df['train_test'] + df['style'] 
    
    return df

def save_img(x):
    img = x['filename']
    old_image_path = os.path.join(old_folder, img)
    new_image_path = os.path.join(x['folder_name'], img)
    shutil.copy(old_image_path, new_image_path)

def organize_dataset(df, images, folder_path, train_ratio):
    # créer le nouveau dossier rangé par style
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    df = train_test(df, images, folder_path, train_ratio)
    
    for folder in list(df.folder_name.unique()):
        if not os.path.exists(folder):
            os.makedirs(folder)
    df.apply(save_img, axis = 1)
    return df
    
recap_csv = organize_dataset(df, images, folder_path, 0.8) 


/home/mathilde/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/mathilde/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [14]:
recap_csv.groupby(['train_test', 'style'])['filename'].count()

train_test                     style        
../data/classif_style2/test/   cubism            27
                               impressionism    179
../data/classif_style2/train/  cubism           111
                               impressionism    720
Name: filename, dtype: int64

In [10]:
df = df[df['filename'].isin(images)]
df['nb_tot'] = df.groupby(['style'])['filename'].transform('count')
df.sort_values(by = 'nb_tot', ascending = False)[['style','nb_tot']].drop_duplicates()[:15]

,style,nb_tot
47769,Impressionism,899.0
33981,Realism,850.0
48245,Romanticism,685.0
65353,Expressionism,566.0
76700,Post-Impressionism,454.0
34378,Art Nouveau (Modern),396.0
6467,Surrealism,374.0
74074,Baroque,353.0
50364,Symbolism,260.0
51288,Rococo,226.0
